# Running Genesis4

In [ ]:
import genesis.version4 as g4

%config InlineBackend.figure_format = 'retina' # Nicer plots

## Create the main input and lattice

In [ ]:
main = g4.MainInput(
    [
        g4.Setup(
            rootname="Example1",
            beamline="FEL",
            gamma0=11357.82,
            delz=0.045,
            nbins=8,
            shotnoise=False,
        ),
        g4.LatticeNamelist(zmatch=9.5),
        g4.Field(power=5000.0, waist_size=3e-05, dgrid=0.0002, ngrid=255),
        g4.Beam(delgam=1.0, current=3000.0, ex=4e-07, ey=4e-07),
        g4.Track(),
    ],
)

lattice = g4.Lattice(
    {
        "D1": g4.Drift(L=0.44),
        "D2": g4.Drift(L=0.24),
        "FEL": g4.Line(elements=["FODO"] * 6),
        "FODO": g4.Line(
            elements=["UND", "D1", "QF", "D2", "UND", "D1", "QD", "D2"],
        ),
        "QD": g4.Quadrupole(L=0.08, k1=-2.0),
        "QF": g4.Quadrupole(L=0.08, k1=2.0),
        "UND": g4.Undulator(aw=0.84853, lambdau=0.015, nwig=266, helical=True),
    }
)

## Run without verbose output 

In [ ]:
G = g4.Genesis4(main, lattice, verbose=False)

In [ ]:
output = G.run()

In [ ]:
G.plot(["beam_xsize", "beam_ysize", "field_xsize", "field_ysize"])

In [ ]:
output.info()

In [ ]:
output["beam_energy"]

We can still access the output in `output.run.output_log`:

In [ ]:
print(output.run.output_log[:200], "...")

## Run with verbose output

This allows you to monitor Genesis4 as it runs.

In [ ]:
G.verbose = True
output = G.run()

## Configure MPI for running on your machine

By default, Genesis4 objects are configured to use 1 processor (`G.nproc = 1`).

To run Genesis4 with MPI, we first need to tell it how many processors to use. To run on 8 cores, we could simply set:
```python
G.nproc = 8
```

A special value of `0` or `None` will tell LUME-Genesis to use all available processor cores on your machine:
```python
G.nproc = 0
```
We'll use that convenient value below.


In [ ]:
G.nproc = 0
output = G.run()

We should see the "MPI-Comm size" as the number of processor cores used by Genesis4 in the output:

In [ ]:
for line in G.output.run.output_log.splitlines():
    if "MPI" in line:
        print(line)

In [ ]:
output.run.to_table()